In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression


# 気候データとホットリンクとナイトレイと曜日ダミーのみからやる


In [2]:
#tsvファイルの読み込み
train=pd.read_csv("./train.tsv",delimiter="\t")
weather=pd.read_csv("./weather.tsv",delimiter="\t")
test=pd.read_csv("./test.tsv",delimiter="\t")
hotlink=pd.read_csv("./hotlink.tsv",delimiter="\t")
nightley=pd.read_csv("./nightley.tsv",delimiter="\t")

#年月日の書き方がバラバラなので揃える
train["datetime"]=pd.DatetimeIndex(train["datetime"].values)
test["datetime"]=pd.DatetimeIndex(test["datetime"].values)
hotlink["datetime"]=pd.DatetimeIndex(hotlink["datetime"].values)
nightley["datetime"]=pd.DatetimeIndex(nightley["datetime"].values)
weather["年月日"]=pd.DatetimeIndex(weather["年月日"].apply(lambda x:x.replace("/","-")).values)


In [3]:
for w in range(7):
    train["isweek"+str(w)]=(pd.DatetimeIndex(train["datetime"]).weekday==w).astype(int)
    test["isweek"+str(w)]=(pd.DatetimeIndex(test["datetime"]).weekday==w).astype(int)
    
for m in range(1,13):
    train["ismonth"+str(m)]=train["datetime"].astype(str).apply(lambda x:x[5:7]).astype(int)==m
    test["ismonth"+str(m)]=test["datetime"].astype(str).apply(lambda x:x[5:7]).astype(int)==m

In [4]:
train.head()

,datetime,park,visitors,isweek0,isweek1,isweek2,isweek3,isweek4,isweek5,isweek6,...,ismonth3,ismonth4,ismonth5,ismonth6,ismonth7,ismonth8,ismonth9,ismonth10,ismonth11,ismonth12
0,2015-01-01,阿寒摩周国立公園,11028,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,2015-01-01,十和田八幡平国立公園,34757,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,2015-01-01,日光国立公園,29714,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,2015-01-01,伊勢志摩国立公園,42652,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,2015-01-01,大山隠岐国立公園,3637,0,0,0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False


In [5]:
#気候情報・地点・公園を全て揃える
weather_useful_cols=[c for c in weather.columns.difference(["年月日","地点","最多風向(16方位)"]) if weather[c].dropna().shape[0]>=10000]
chitens=weather["地点"].drop_duplicates().values
parks=train["park"].drop_duplicates().values

#hotlinkの一覧も作る
hotlink_domain=hotlink["domain"].drop_duplicates().values
hotlink_keywords=hotlink["keyword"].drop_duplicates().values

In [6]:
hotlinkdf=pd.DataFrame(index=np.arange(len(hotlink["datetime"].drop_duplicates().values)))
hotlinkdf["datetime"]=hotlink["datetime"].drop_duplicates().values
for domain in hotlink_domain:
    for keyword in hotlink_keywords:
        df_=hotlink[(hotlink["domain"]==domain)&(hotlink["keyword"]==keyword)][["datetime","count"]]
        df_.columns=["datetime",domain[:2]+keyword]
        hotlinkdf=pd.merge(hotlinkdf,df_,on="datetime",how="left")

In [7]:
weather_useful_cols

['10分間降水量の最大(mm)',
 '平均気温(℃)',
 '平均風速(m/s)',
 '日照時間(時間)',
 '最低気温(℃)',
 '最大瞬間風速(m/s)',
 '最大風速(m/s)',
 '最高気温(℃)',
 '降水量の合計(mm)']

In [8]:
chitens

array(['十和田', '大山', '大田', '日光', '渡嘉敷', '熊本', '釧路', '青森', '高森', '鳥羽',
       '鹿児島', '鹿角'], dtype=object)

In [9]:
parks

array(['阿寒摩周国立公園', '十和田八幡平国立公園', '日光国立公園', '伊勢志摩国立公園', '大山隠岐国立公園',
       '阿蘇くじゅう国立公園', '霧島錦江湾国立公園', '慶良間諸島国立公園'], dtype=object)

In [10]:
hotlinkdf.head()

,datetime,tw国立公園,twアオウミガメ,tw阿波連岬園地,tw鬼怒川,tw稲崎園地,tw東照宮,twミヤマキリシマ,twノカイドウ,tw栗野八幡地獄,...,bbオンネトー,bbアカウミガメ,bb鶴見岳,bbダイセンキャラボク,bb中禅寺湖,bbオキサンショウウオ,bb出雲大社,bb隠岐自然回帰の森,bb浄土ヶ浦,bbハマユウ
0,2015-01-01,148,1,0,95,0,174,3,0,0,...,0,0,0,0,0,0,16,0,0,0
1,2015-01-02,171,4,0,57,0,88,0,0,0,...,0,0,0,0,1,0,20,0,0,0
2,2015-01-03,164,1,0,65,0,92,2,0,0,...,0,0,0,0,0,0,24,0,0,0
3,2015-01-04,133,6,0,31,0,62,1,0,0,...,0,0,0,0,1,0,13,0,0,0
4,2015-01-05,155,1,0,40,0,73,0,0,0,...,0,0,0,0,1,0,13,0,0,0


In [11]:
#[地点_気候]というカラムを追加していく
for chiten in chitens:
    df=weather[weather["地点"]==chiten]
    for col in weather_useful_cols:
        df_=df[["年月日",col]]
        df_.columns=["datetime",chiten+"_"+col]
        train=pd.merge(train,df_,on="datetime",how="left")
        test=pd.merge(test,df_,on="datetime",how="left")

train=pd.merge(train,hotlinkdf,on="datetime",how="left")
test=pd.merge(test,hotlinkdf,on="datetime",how="left")

In [12]:
train.head()

,datetime,park,visitors,isweek0,isweek1,isweek2,isweek3,isweek4,isweek5,isweek6,...,bbオンネトー,bbアカウミガメ,bb鶴見岳,bbダイセンキャラボク,bb中禅寺湖,bbオキサンショウウオ,bb出雲大社,bb隠岐自然回帰の森,bb浄土ヶ浦,bbハマユウ
0,2015-01-01,阿寒摩周国立公園,11028,0,0,0,1,0,0,0,...,0,0,0,0,0,0,16,0,0,0
1,2015-01-01,十和田八幡平国立公園,34757,0,0,0,1,0,0,0,...,0,0,0,0,0,0,16,0,0,0
2,2015-01-01,日光国立公園,29714,0,0,0,1,0,0,0,...,0,0,0,0,0,0,16,0,0,0
3,2015-01-01,伊勢志摩国立公園,42652,0,0,0,1,0,0,0,...,0,0,0,0,0,0,16,0,0,0
4,2015-01-01,大山隠岐国立公園,3637,0,0,0,1,0,0,0,...,0,0,0,0,0,0,16,0,0,0


In [13]:
#公園ごとにtrain_,test_を作る。何も考えずにAdaBoostを使う。
#all_testに答えをどんどん蓄積
all_test=pd.DataFrame()
for park in parks:
    if park=="日光国立公園":
        continue
    train_=train[train["park"]==park].fillna(0)
    test_=test[test["park"]==park].fillna(0).reset_index()
    x_col=train_.columns.difference(["datetime","park","visitors"])
    y_col="visitors"
    test_[y_col]=AdaBoostRegressor().fit(train_[x_col].values,train_[y_col].values).predict(test_[x_col])
    all_test=all_test.append(test_[["index",y_col]])
    
train_=train[train["park"]=="日光国立公園"].fillna(0)
test_=test[test["park"]=="日光国立公園"].fillna(0).reset_index()
nightley_=nightley.append(pd.DataFrame([np.datetime64("2017-12-31"),np.mean(nightley["Japan_count"]),np.mean(nightley["Foreign_count"])],index=nightley.columns).T)
nightley_["datetime"]=pd.DatetimeIndex(nightley_["datetime"])

train_=pd.merge(train_,nightley_,on="datetime")
test_=pd.merge(test_,nightley_,on="datetime",how="left")
x_col=train_.columns.difference(["datetime","park","visitors"])
y_col="visitors"
test_[y_col]=AdaBoostRegressor().fit(train_[x_col].values,train_[y_col].values).predict(test_[x_col])
all_test=all_test.append(test_[["index","visitors"]])


In [14]:
#提出
all_test.sort_values(by="index")[["index","visitors"]].to_csv("test4.tsv",sep="\t",header=None,index=None)